In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import pandas as pd
import librosa
import os, re, glob
from sklearn.model_selection import GroupShuffleSplit
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow as tf
import os

In [4]:
# ====== MFCC sekvens (bevarer tidsaksen) ======


# --- Vinduer: 400 ms med 50% overlap ---
def slice_to_windows(x, sr, win_ms=400, hop_ms=200):
    win = int(round(sr * win_ms/1000))
    hop = int(round(sr * hop_ms/1000))
    if len(x) < win:
        x = np.pad(x, (0, win - len(x)))
    starts = np.arange(0, len(x) - win + 1, hop, dtype=int)
    windows = [x[s:s+win] for s in starts]
    return np.stack(windows) if windows else np.empty((0, win), dtype=np.float32)

# --- MFCC-sekvens: EI-match (ingen deltas) ---
import numpy as np, librosa

def mfcc_sequence_EI(signal, sample_rate,
                     n_mfcc=13, n_mels=32,
                     win_length_ms=25, hop_length_ms=20,
                     fmin=80, fmax=None,
                     n_fft=512, pre_emph=0.98,
                     center=False, add_deltas=False):
    x = np.asarray(signal, dtype=np.float32)
    if x.ndim == 2:
        x = x.mean(axis=1)

    # pre-emphasis 0.98
    x = np.append(x[0], x[1:] - pre_emph * x[:-1])

    n_fft = int(n_fft)
    hop   = int(round(sample_rate * hop_length_ms / 1000))
    win   = int(round(sample_rate * win_length_ms / 1000))
    if fmax is None:
        fmax = sample_rate / 2

    # mel power → dB (ref=1.0 for absolut skala)
    S = librosa.feature.melspectrogram(
        y=x, sr=sample_rate, n_fft=n_fft, hop_length=hop, win_length=win,
        window="hann", center=center, power=2.0,
        n_mels=n_mels, fmin=fmin, fmax=fmax, htk=True
    )
    S_db = librosa.power_to_db(S, ref=1.0, top_db=80.0)

    mfcc = librosa.feature.mfcc(S=S_db, sr=sample_rate, n_mfcc=n_mfcc)  # (n_mfcc, T)

    feats = [mfcc]
    if add_deltas:
        d1 = librosa.feature.delta(mfcc, order=1, width=9, mode="nearest")
        d2 = librosa.feature.delta(mfcc, order=2, width=9, mode="nearest")
        feats += [d1, d2]
    X = np.concatenate(feats, axis=0).T  # (T, F)

    # "Normalization window size = 151": T<<151 → per-vindue CMVN ~ global.
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, keepdims=True) + 1e-6
    X = (X - mu) / sd

    return X.astype(np.float32)

# ====== Læs én WAV → (X, y, car, files) ======
def process_wav_mfcc_conv1d(file_path, label_int, car_id,
                            sr_expected=None,
                            win_ms=400, hop_ms=200,
                            n_mfcc=13, n_mels=32,
                            win_length_ms=25, hop_length_ms=20):
    sr, data = wavfile.read(file_path)
    x = np.asarray(data, dtype=np.float32)
    if x.ndim == 2:
        x = x.mean(axis=1)
    mx = np.max(np.abs(x))
    if mx > 0: x = x / mx
    if sr_expected is not None and sr != sr_expected:
        # (tilpas evt. med librosa.resample)
        raise ValueError(f"Sample rate mismatch: {sr} != {sr_expected}")

    segments = slice_to_windows(x, sr, win_ms, hop_ms)  # (N, samples)
    X_list = []
    for seg in segments:
        X_seq = mfcc_sequence_EI(seg, sample_rate=sr,
                              n_mfcc=n_mfcc, n_mels=n_mels,
                              win_length_ms=win_length_ms, hop_length_ms=hop_length_ms,
                              add_deltas=False, center=False)  # (T,F)
        X_list.append(X_seq)

    if not X_list:
        return (np.empty((0,1,1), np.float32),
                np.empty((0,), np.int32),
                np.empty((0,), object),
                np.empty((0,), object))

    X = np.stack(X_list)                          # (N, T, F)
    y = np.full((X.shape[0],), label_int, np.int32)
    car = np.full((X.shape[0],), car_id, object)
    files = np.array([os.path.basename(file_path)] * X.shape[0], object)
    return X, y, car, files

# ====== Scan mapper og byg hele datasættet ======
def build_dataset_conv1d(root_dir, sr_expected=None):
    # Find NORMAL/FAULTY
    normal_paths = glob.glob(os.path.join(root_dir, "NORMAL", "**", "*.wav"), recursive=True)
    faulty_paths = glob.glob(os.path.join(root_dir, "FAULTY", "**", "*.wav"), recursive=True)

    label_map = {"NORMAL": 0, "FAULTY": 1}

    # Hjælper: udtræk car_id fra filnavn (tilpas regex til dit mønster)
    car_re = re.compile(r"(car\d+)", re.IGNORECASE)

    X_list, y_list, g_list, f_list = [], [], [], []

    for label_name, paths in [("NORMAL", normal_paths), ("FAULTY", faulty_paths)]:
        for p in paths:
            m = car_re.search(os.path.basename(p))
            car_id = (m.group(1) if m else "UNKNOWN").upper()
            Xi, yi, gi, fi = process_wav_mfcc_conv1d(
                p, label_map[label_name], car_id,
                sr_expected=sr_expected,
                win_ms=400, hop_ms=200,
                n_mfcc=13, n_mels=32,
                win_length_ms=25, hop_length_ms=20
            )
            if Xi.shape[0] == 0:
                continue
            X_list.append(Xi)
            y_list.append(yi)
            g_list.append(gi)
            f_list.append(fi)

    X = np.concatenate(X_list, axis=0) if X_list else np.empty((0,1,1), np.float32)
    y = np.concatenate(y_list, axis=0) if y_list else np.empty((0,), np.int32)
    groups = np.concatenate(g_list, axis=0) if g_list else np.empty((0,), object)
    files = np.concatenate(f_list, axis=0) if f_list else np.empty((0,), object)
    return X, y, groups, files, label_map

In [ ]:
root_dir = r"C:\Users\nusse\Desktop\EH5TinyML\soundovertcp\Server\out"
X, y, groups, files, label_map = build_dataset_conv1d(root_dir, sr_expected=None)

print("X:", X.shape)     # (N, T, F) 
print("y:", y.shape)
print("unique cars:", pd.unique(groups))

X: (19665, 19, 13)
y: (19665,)
unique cars: ['CAR01' 'CAR02' 'CAR03' 'CAR04']


In [6]:
from sklearn.model_selection import train_test_split

def stratified_per_car_split(groups, y, test_size=0.2, random_state=42):
    """
    Returnér tr_idx, te_idx sådan at:
      - HVER bil (group) er til stede i både train og test
      - FOR HVER bil er test-delen stratificeret på y (NORMAL/FAULTY)
    """
    groups = np.asarray(groups)
    y = np.asarray(y)
    all_idx = np.arange(len(y))

    te_mask = np.zeros(len(y), dtype=bool)

    rng = np.random.RandomState(random_state)
    for car in np.unique(groups):
        idx_car = all_idx[groups == car]
        y_car = y[idx_car]
        # stratificeret split INDEN i denne bil
        _, te_idx_car = train_test_split(
            idx_car, test_size=test_size, stratify=y_car, random_state=rng.randint(0, 10**9)
        )
        te_mask[te_idx_car] = True

    te_idx = np.where(te_mask)[0]
    tr_idx = np.where(~te_mask)[0]
    return tr_idx, te_idx

In [7]:
# lav split
tr_idx, te_idx = stratified_per_car_split(groups, y, test_size=0.2, random_state=42)

# lav dine datasæt
Xtr, Xte = X[tr_idx], X[te_idx]
ytr, yte = y[tr_idx], y[te_idx]
groups_tr, groups_te = groups[tr_idx], groups[te_idx]

# sanity-check: hver bil findes i begge sæt, og test har begge labels pr. bil
import pandas as pd
print("Train cars:", np.unique(groups_tr))
print("Test  cars:", np.unique(groups_te))

print("TEST per car & label:")
df_te = pd.DataFrame({'car': groups_te, 'y': yte})
print(df_te.groupby(['car','y']).size().unstack(fill_value=0))

Train cars: ['CAR01' 'CAR02' 'CAR03' 'CAR04']
Test  cars: ['CAR01' 'CAR02' 'CAR03' 'CAR04']
TEST per car & label:
y        0    1
car            
CAR01  477  479
CAR02  480  479
CAR03  477  587
CAR04  477  479


In [8]:
# Split train -> train + val, stadig stratificeret pr. bil
from sklearn.model_selection import train_test_split
tr2_idx, val_idx = train_test_split(
    np.arange(len(Xtr)),
    test_size=0.2,
    stratify=ytr,
    random_state=123
)

X_tr, X_val = Xtr[tr2_idx], Xtr[val_idx]
y_tr, y_val = ytr[tr2_idx], ytr[val_idx]
groups_tr2, groups_val = groups_tr[tr2_idx], groups_tr[val_idx]

print("Train:", X_tr.shape, "Val:", X_val.shape, "Test:", Xte.shape)

Train: (12584, 19, 13) Val: (3146, 19, 13) Test: (3935, 19, 13)


In [9]:


# ====== 0) Sikr korrekt format og one-hot ======
assert X_tr.ndim == 3 and X_tr.shape[2] == 13, f"forvent (N,T,13), fik {X_tr.shape}"
assert X_val.ndim == 3 and X_val.shape[2] == 13
num_classes = int(np.unique(y_tr).size)
assert num_classes == 2, "den her skabelon er sat op til NORMAL/FAULTY (2 klasser)"

# one-hot labels (EI bruger categorical_crossentropy + softmax)
y_tr_oh  = tf.one_hot(y_tr.astype(np.int32), depth=num_classes)
y_val_oh = tf.one_hot(y_val.astype(np.int32), depth=num_classes)

# Flatten to (input_length,) for EI-style InputLayer + Reshape til (T,13)
T, F = X_tr.shape[1], X_tr.shape[2]    # fx (19, 13)
input_length = T * F                   # fx 247
X_tr_flat = X_tr.reshape((-1, input_length)).astype(np.float32)
X_val_flat = X_val.reshape((-1, input_length)).astype(np.float32)

# ====== 1) SpecAugment-lignende mapper (time/freq mask) ======
# (meget letvægts version; samme idé som EI’s SpecAugment-parametre)
def specaugment_mapper(mF=1, Fmax=4, mT=1, Tmax=1):
    # forventer (input_length,) -> vi reshaper til (T,13) inde i mappet
    def _map(x, y):
        x = tf.reshape(x, (T, F))  # (T,13)

        # freq masks
        for _ in range(mF):
            f = tf.random.uniform([], minval=0, maxval=Fmax+1, dtype=tf.int32)
            f0 = tf.random.uniform([], minval=0, maxval=F - f + 1, dtype=tf.int32)
            mask = tf.concat([
                tf.ones((T, f0), dtype=x.dtype),
                tf.zeros((T, f), dtype=x.dtype),
                tf.ones((T, F - f0 - f), dtype=x.dtype)
            ], axis=1)
            x = x * mask

        # time masks
        for _ in range(mT):
            t = tf.random.uniform([], minval=0, maxval=Tmax+1, dtype=tf.int32)
            t0 = tf.random.uniform([], minval=0, maxval=T - t + 1, dtype=tf.int32)
            mask = tf.concat([
                tf.ones((t0, F), dtype=x.dtype),
                tf.zeros((t, F), dtype=x.dtype),
                tf.ones((T - t0 - t, F), dtype=x.dtype)
            ], axis=0)
            x = x * mask

        # tilbage til (input_length,)
        x = tf.reshape(x, (input_length,))
        return x, y
    return _map

# ====== 2) tf.data pipelines ======
BATCH_SIZE = 32
ENSURE_DETERMINISM = False
LEARNING_RATE = 0.005
EPOCHS = 100

train_ds = tf.data.Dataset.from_tensor_slices((X_tr_flat, y_tr_oh))
val_ds   = tf.data.Dataset.from_tensor_slices((X_val_flat, y_val_oh))

if not ENSURE_DETERMINISM:
    train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE*4, reshuffle_each_iteration=True)

# GaussianNoise ligger i modellen; her laver vi SpecAugment på træning
sa = specaugment_mapper(mF=1, Fmax=4, mT=1, Tmax=1)  # svarer ca. til EI’s params i dit snippet
train_ds = train_ds.map(sa, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=False).prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.batch(BATCH_SIZE, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

# ====== 3) Model (EI-style) ======
# Bemærk: vi følger dit EI-snippet tæt: GaussianNoise -> Reshape -> Conv1D×3 -> MaxPool -> Flatten -> Dropout -> Dense softmax
model = models.Sequential([
    layers.InputLayer(input_shape=(input_length,)),
    layers.GaussianNoise(stddev=0.2),                     # EI: GaussianNoise på flattened input
    layers.Reshape((T, F)),                                # -> (T,13)
    layers.Conv1D(16, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling1D(pool_size=2, strides=2, padding='same'),
    layers.Conv1D(32, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling1D(pool_size=2, strides=2, padding='same'),
    layers.Conv1D(64, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling1D(pool_size=2, strides=2, padding='same'),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, name='y_pred', activation='softmax'),
])

opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# (valgfri) callbacks ala EI
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                         patience=5, min_lr=1e-5, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                     restore_best_weights=True)
]

# ====== 4) Train & eval ======
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, verbose=2, callbacks=callbacks)
val_loss, val_acc = model.evaluate(val_ds, verbose=0)
print("VAL acc:", val_acc)

Epoch 1/100


C:\Users\nusse\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


394/394 - 2s - 6ms/step - accuracy: 0.7923 - loss: 0.4008 - val_accuracy: 0.9282 - val_loss: 0.1766 - learning_rate: 0.0050
Epoch 2/100
394/394 - 1s - 3ms/step - accuracy: 0.9105 - loss: 0.2114 - val_accuracy: 0.9720 - val_loss: 0.0761 - learning_rate: 0.0050
Epoch 3/100
394/394 - 1s - 3ms/step - accuracy: 0.9336 - loss: 0.1609 - val_accuracy: 0.9739 - val_loss: 0.0795 - learning_rate: 0.0050
Epoch 4/100
394/394 - 1s - 3ms/step - accuracy: 0.9429 - loss: 0.1499 - val_accuracy: 0.9812 - val_loss: 0.0569 - learning_rate: 0.0050
Epoch 5/100
394/394 - 1s - 3ms/step - accuracy: 0.9459 - loss: 0.1331 - val_accuracy: 0.9800 - val_loss: 0.0605 - learning_rate: 0.0050
Epoch 6/100
394/394 - 1s - 3ms/step - accuracy: 0.9527 - loss: 0.1220 - val_accuracy: 0.9882 - val_loss: 0.0417 - learning_rate: 0.0050
Epoch 7/100
394/394 - 1s - 3ms/step - accuracy: 0.9509 - loss: 0.1229 - val_accuracy: 0.9851 - val_loss: 0.0462 - learning_rate: 0.0050
Epoch 8/100
394/394 - 1s - 3ms/step - accuracy: 0.9558 - los

In [ ]:
from keras2c import k2c
model_inf = models.Sequential([
    layers.InputLayer(input_shape=(input_length,)),
    layers.Reshape((T, F)),                                
    layers.Conv1D(16, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling1D(pool_size=2, strides=2, padding='same'),
    layers.Conv1D(32, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling1D(pool_size=2, strides=2, padding='same'),
    layers.Conv1D(64, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling1D(pool_size=2, strides=2, padding='same'),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, name='y_pred', activation='softmax'),
])
model_inf.set_weights(model.get_weights())
k2c(model_inf, "mymodel", num_tests=10)

All checks passed
Gathering Weights
Writing layer  keras_tensor_82
Writing layer  keras_tensor_83
Writing layer  keras_tensor_84
Writing layer  keras_tensor_85
Writing layer  keras_tensor_86
Writing layer  keras_tensor_87
Writing layer  keras_tensor_88
Writing layer  keras_tensor_89
Writing layer  keras_tensor_90
Writing layer  keras_tensor_91
astyle not found, mymodel.h and mymodel.c will not be auto-formatted
Writing tests
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


C:\Users\nusse\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
astyle not found, mymodel_test_suite.c will not be auto-formatted
Done 
C code is in 'mymodel.c' with header file 'mymodel.h' 
Tests are in 'mymodel_test_suite.c' 



AssertionError: The following errors were found:
Layer type 'GaussianNoise' is not supported at this time.
